In [3]:
'''
REGRESSION:
    Goal: Predict Productivity Score based on features
    
'''

'\nREGRESSION:\n    Goal: Predict Productivity Score based on features\n\n'

In [4]:
import pandas as pd

df = pd.read_csv('./datasets/social_media_vs_productivity_cleaned.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'age', 'gender', 'job_type', 'daily_social_media_time',
       'social_platform_preference', 'number_of_notifications',
       'work_hours_per_day', 'perceived_productivity_score',
       'actual_productivity_score', 'stress_level', 'sleep_hours',
       'screen_time_before_sleep', 'breaks_during_work', 'uses_focus_apps',
       'has_digital_wellbeing_enabled', 'coffee_consumption_per_day',
       'days_feeling_burnout_per_month', 'weekly_offline_hours',
       'job_satisfaction_score'],
      dtype='object')

In [9]:
# features that don't have anything to do with productivity, as well as the output.

input_invalid_columns =  ['social_platform_preference', 'perceived_productivity_score', 'actual_productivity_score','Unnamed: 0']

In [10]:
features = df.drop(columns=input_invalid_columns)

features.head()

,age,gender,job_type,daily_social_media_time,number_of_notifications,work_hours_per_day,stress_level,sleep_hours,screen_time_before_sleep,breaks_during_work,uses_focus_apps,has_digital_wellbeing_enabled,coffee_consumption_per_day,days_feeling_burnout_per_month,weekly_offline_hours,job_satisfaction_score
0,56,0,0,4.180940,61,6.753558,4.0,5.116546,0.419102,8,0,0,4,11,21.927072,6.336688
1,46,0,1,3.249603,59,9.169296,7.0,5.103897,0.671519,7,1,1,2,25,0.000000,3.412427
2,32,0,2,0.000000,57,7.910952,4.0,8.583222,0.624378,0,1,0,3,17,10.322044,2.474944
3,60,1,0,0.000000,59,6.355027,6.0,6.052984,1.204540,1,0,0,0,4,23.876616,1.733670
4,25,0,3,0.000000,66,6.214096,7.0,5.405706,1.876254,1,0,1,1,30,10.653519,9.693060


In [8]:
output = df['actual_productivity_score']

In [11]:
import numpy as np

input_features = features.to_numpy()
outputs = output.to_numpy()

In [20]:
from sklearn.model_selection import train_test_split

# First split: train vs test
X_train_full, X_test, y_train_full, y_test = train_test_split(
    input_features, outputs, test_size=0.2, random_state=42
)

# Second split: train vs validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42
)


In [21]:
# shapes 

print(f"X_train shape {X_train.shape}")
print(f"X_test shape {X_test.shape}")
print(f"y_train shape {y_train.shape}")
print(f"y_test shape {y_test.shape}")

print("validation:")
print(f"X_val shape {X_val.shape}")
print(f"y_val shape {y_val.shape}")

X_train shape (18000, 16)
X_test shape (6000, 16)
y_train shape (18000,)
y_test shape (6000,)
validation:
X_val shape (6000, 16)
y_val shape (6000,)


In [ ]:
import rerf 